In [ ]:
# Splunk Pre-Installation Checks & System Configuration

# -------------------------------------------------------------
# ✅ Check Minimum Hardware Requirements
# -------------------------------------------------------------

# Check OS version
cat /etc/os-release

# Check Disk Partition
df -kh
cd /opt/splunk || echo "/opt/splunk not found"

# Get CPU Details
lscpu

# Get Memory Details
free -h

# -------------------------------------------------------------
# 👤 Create Splunk User, Directories & Permissions
# -------------------------------------------------------------

# Create user & group
sudo groupadd splunk || echo "group exists"
sudo useradd splunk -g splunk || echo "user exists"

# Create directory
sudo mkdir -p /opt/splunk

# Change ownership
sudo chown -R splunk:splunk /opt/splunk/

# Verify ownership
ls -ld /opt/splunk

# Install wget
sudo yum install wget -y

# -------------------------------------------------------------
# 🔒 Verify Required Ports are Open
# -------------------------------------------------------------
# Ports: 8000, 8089, 8088, 9997, 8065, 8181, 9887, 8191, 514, 9777

sudo netstat -tuln | grep -E ':8000|:8089|:8088|:9997|:8065|:8181|:9887|:8191|:514|:9777' || echo "No ports listening"
sudo ss -tuln | grep -E ':8000|:8089|:8088|:9997|:8065|:8181|:9887|:8191|:514|:9777' || echo "No ports listening"

# -------------------------------------------------------------
# ⚙️ Check Transparent Huge Pages (THP) Status
# -------------------------------------------------------------
cat /sys/kernel/mm/transparent_hugepage/enabled
cat /sys/kernel/mm/transparent_hugepage/defrag

# -------------------------------------------------------------
# 📈 Check ulimit Values
# -------------------------------------------------------------
ulimit -a
sudo cat /etc/security/limits.conf | tail -n 20

# -------------------------------------------------------------
# ⏰ Check NTP Sync Status
# -------------------------------------------------------------
ntpstat 

Output:
|| echo "NTP not installed"

# -------------------------------------------------------------
# 🖥️ Change Hostname
# -------------------------------------------------------------
# Example: change with Splunk Component name
#!sudo hostnamectl set-hostname splunk-indexer-01

# -------------------------------------------------------------
# 🔧 Disable THP
# -------------------------------------------------------------
# Reference: https://docs.splunk.com/Documentation/Splunk/9.2.2/ReleaseNotes/SplunkandTHP

# ---- For RHEL ≤ 9.4 ----
# Run the following manually:
"""
sudo su
cd /etc/init.d/
vi disable_thp
(paste script below)

#!/bin/bash
### BEGIN INIT INFO
# Provides:          disable-thp
# Required-Start:    $local_fs
# Required-Stop:
# X-Start-Before:    couchbase-server
# Default-Start:     2 3 4 5
# Default-Stop:      0 1 6
# Short-Description: Disable THP
# Description:       Disables transparent huge pages (THP)
### END INIT INFO

case $1 in
  start)
    if [ -d /sys/kernel/mm/transparent_hugepage ]; then
      thp_path=/sys/kernel/mm/transparent_hugepage
    elif [ -d /sys/kernel/mm/redhat_transparent_hugepage ]; then
      thp_path=/sys/kernel/mm/redhat_transparent_hugepage
    else
      return 0
    fi

    echo 'never' > ${thp_path}/enabled
    echo 'never' > ${thp_path}/defrag

    re='^[0-1]+$'
    if [[ $(cat ${thp_path}/khugepaged/defrag) =~ $re ]]
    then
      echo 0  > ${thp_path}/khugepaged/defrag
    else
      echo 'no' > ${thp_path}/khugepaged/defrag
    fi
    ;;
esac
"""

# Make it executable
#!sudo chmod 755 /etc/init.d/disable_thp
#!sudo chkconfig --add disable_thp
#!sudo reboot

# ---- For RHEL ≥ 9.6 ----
"""
sudo tee /etc/systemd/system/disable-thp.service > /dev/null << EOF
[Unit]
Description=Disable Transparent Huge Pages and defrag
After=network.target

[Service]
Type=oneshot
ExecStart=/bin/bash -c 'echo never > /sys/kernel/mm/transparent_hugepage/enabled; echo never > /sys/kernel/mm/transparent_hugepage/defrag'

[Install]
WantedBy=multi-user.target
EOF

sudo systemctl daemon-reexec
sudo systemctl daemon-reload
sudo systemctl enable disable-thp
sudo reboot
"""

# -------------------------------------------------------------
# 🔧 Increase ulimit Values
# -------------------------------------------------------------
# Edit limits.conf manually:
"""
sudo vi /etc/security/limits.conf
"""

# Add the following values:
"""
* soft nofile 64000
* hard nofile unlimited

* soft nproc 16000
* hard nproc unlimited

* soft data 8000000
* hard data unlimited

* soft fsize unlimited
* hard fsize unlimited
"""

# Reference: https://docs.splunk.com/Documentation/Splunk/9.2.2/Installation/Systemrequirements
# After editing:
#!sudo reboot
